In [1]:
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, output_notebook, ColumnDataSource
from bokeh.models import LinearColorMapper, ColorBar, CustomJS, Rect, HoverTool
from bokeh.layouts import column

%matplotlib notebook

output_notebook()

Loading BokehJS ...

In [2]:
#match the paths to the binary files containing data to the file locations on machine 
field400 = np.load('/Users/danslaughter/Downloads/Field_400_norm_array.npy')
x = np.load('/Users/danslaughter/Downloads/times_array.npy')
y = np.load('/Users/danslaughter/Downloads/intensities_array.npy')

In [3]:
TOOLS = "crosshair, pan, wheel_zoom, zoom_in, zoom_out, box_zoom, undo, redo, reset, save"

p1 = figure(tools = TOOLS, plot_width = 600, plot_height = 400, 
           x_range = (np.amin(x), np.amax(x)), y_range = (np.amin(y), np.amax(y)))

p1.image(image = [field400], x = [np.amin(x)], y = [np.amin(y)], dw = [np.amax(x) - np.amin(x)],
         dh = [np.amax(y) - np.amin(y)], palette = "Viridis256")

color_mapper = LinearColorMapper(palette = "Viridis256", low = 0.0, high = 1.0)
color_bar = ColorBar(color_mapper = color_mapper,
                     label_standoff = 12, border_line_color = None, location = (0,0))

p1.add_layout(color_bar, 'right')

p1.title.text = "9th Harmonic Intensity (Pan/Zoom Here)"
p1.title.text_font_size = "18px"
p1.title.text_font = "times"
p1.title.text_font_style = "bold"
p1.title.align = "left"

p1.xaxis.axis_label = "Time (fs)"
p1.xaxis.axis_label_text_font = "times"
p1.xaxis.axis_label_text_font_size = "14px"
p1.xaxis.axis_label_text_font_style = "normal"

p1.yaxis.axis_label = "Peak Intensity (a.u.)"
p1.yaxis.axis_label_text_font = "times"
p1.yaxis.axis_label_text_font_size = "14px"
p1.yaxis.axis_label_text_font_style = "normal"

source = ColumnDataSource({'x': [], 'y': [], 'width': [], 'height': []})

jscode="""
    var data = source.data;
    var start = cb_obj.start;
    var end = cb_obj.end;
    data['%s'] = [start + (end - start) / 2];
    data['%s'] = [end - start];
    source.change.emit();
"""

p1.x_range.callback = CustomJS(args = dict(source = source), code = jscode % ('x', 'width'))
p1.y_range.callback = CustomJS(args = dict(source = source), code = jscode % ('y', 'height'))

In [4]:
p2 = figure(tools = TOOLS, plot_width = 600, plot_height = 400, 
           x_range = (np.amin(x), np.amax(x)), y_range = (np.amin(y), np.amax(y)))

p2.image(image = [field400], x = [np.amin(x)], y = [np.amin(y)], dw = [np.amax(x) - np.amin(x)],
         dh = [np.amax(y) - np.amin(y)], palette = "Viridis256")

color_mapper = LinearColorMapper(palette = "Viridis256", low = 0.0, high = 1.0)
color_bar = ColorBar(color_mapper = color_mapper,
                     label_standoff = 12, border_line_color = None, location = (0,0))

p2.add_layout(color_bar, 'right')

p2.title.text = "9th Harmonic Intensity (View Pan/Zoom Window Here)"
p2.title.text_font_size = "18px"
p2.title.text_font = "times"
p2.title.text_font_style = "bold"
p2.title.align = "left"

p2.xaxis.axis_label = "Time (fs)"
p2.xaxis.axis_label_text_font = "times"
p2.xaxis.axis_label_text_font_size = "14px"
p2.xaxis.axis_label_text_font_style = "normal"

p2.yaxis.axis_label = "Peak Intensity (a.u.)"
p2.yaxis.axis_label_text_font = "times"
p2.yaxis.axis_label_text_font_size = "14px"
p2.yaxis.axis_label_text_font_style = "normal"

In [5]:
rect = Rect(x = 'x', y = 'y', width = 'width', height = 'height', fill_alpha = 0.1,
            line_color = 'black', fill_color = 'black')

p2.add_glyph(source, rect)

layout = column([p1, p2])

show(layout)

#in this interaction, when panning and zooming in the top figure,
#the location of the panned or zoomed view window is shown in the bottom figure